# Prepear test dataset
Load word data for error precision test
Format - {word}{several spaces}{answer-words separeted by comma}

In [159]:
!pip install --upgrade pip

In [160]:
!pip install pandas
import pandas as pd

In [161]:
error_precision_test_incorrect_file = open('../data/test_sample_incorrect.txt', 'r', encoding="utf8")
error_precision_test_incorrect_lines = error_precision_test_incorrect_file.readlines()
error_precision_splitted_test_incorrect_lines = list(map(lambda x: x.split(maxsplit=1), error_precision_test_incorrect_lines))
error_precision_test_incorrect_words = list(map(lambda x: x[0], error_precision_splitted_test_incorrect_lines))
error_precision_test_incorrect_answers = list(map(lambda x: list(map(lambda y: y.strip(), x[1].split(', '))), error_precision_splitted_test_incorrect_lines))
error_precision_test_df = pd.DataFrame(error_precision_test_incorrect_words, columns=["test_word"])
error_precision_test_df["answers"] = error_precision_test_incorrect_answers

In [162]:
error_precision_test_df.head()

,test_word,answers
0,эктренном,[экстренном]
1,синусовго,[синусового]
2,даным,"[данным, данный, данные]"
3,шнутография,[шунтография]
4,сникопальных,[синкопальных]


In [163]:
error_precision_test_word_list = error_precision_test_df["test_word"].tolist()

In [164]:
error_precision_test_answers = error_precision_test_df["answers"].tolist()

Load word list for lexical precision test

In [165]:
with open("../data/lexical_word_list.txt") as file:
    lexical_precision_test_word_list = file.readlines()
    lexical_precision_test_word_list = [line.rstrip() for line in lexical_precision_test_word_list]

In [166]:
lexical_precision_test_word_list

['выборгский',
 'отказываться',
 'нитропрепарат',
 'предсердие',
 'диссоциация',
 'кольцевой',
 'отрыв',
 'вовлечение',
 'мелкопузырчатый',
 'госпитализировать',
 'фтизиатр',
 'проксимальный',
 'приступ',
 'рецидивировать',
 'беспокоить',
 'значимый',
 'одномоментный',
 'течение',
 'постепенной',
 'трафика',
 'левый',
 'область',
 'окклюзия',
 'каждый',
 'аторвастатина',
 'проникающий',
 'слух',
 'отрицательный',
 'заднее',
 'диафрагмальный',
 'физическая',
 'реваскуляризация',
 'очевидный',
 'умеренный',
 'самостоятельный',
 'визуализация',
 'неселективный',
 'септопластика',
 'присутствие',
 'соустье',
 'кировский',
 'ноющая',
 'постоять',
 'стенка',
 'укоротить',
 'назначить',
 'госпитализация',
 'возобновление',
 'одеть',
 'охранник',
 'дексона',
 'систолический',
 'верхушечно-перегородочный',
 'расширение',
 'равнозначный',
 'предприятие',
 'ночь',
 'вредность',
 'транспортировка',
 'март',
 'гипотиазид',
 'скрытый',
 'расстройство',
 'ломота',
 'подготовить',
 'помощь',
 'большая

In [167]:
def compute_error_precision(original_word_list, corrected_word_list, answer_word_list):
    words_number = len(corrected_word_list)
    correct_words_number = 0
    print("Error precision")
    print("original_word_list --- corrected_word --- answer_word_list")
    for i, corrected_word in enumerate(corrected_word_list):
        print(f"{original_word_list[i]} --- {corrected_word} --- {answer_word_list[i]}")
        if corrected_word in answer_word_list[i]:
            correct_words_number += 1
    print(f"Right corrected words count - {correct_words_number} of {words_number} total")
    return correct_words_number/words_number

In [168]:
def compute_lexical_precision(original_word_list, corrected_word_list):
    words_number = len(corrected_word_list)
    correct_words_number = 0
    print("Lexical precision")
    print("original_word_list --- corrected_word")
    for i, corrected_word in enumerate(corrected_word_list):
        print(f"{original_word_list[i]} --- {corrected_word}")
        if corrected_word == original_word_list[i]:
            correct_words_number += 1
    print(f"Right corrected words count - {correct_words_number} of {words_number} total")
    return correct_words_number/words_number

In [169]:
!pip install tqdm
from tqdm import tqdm

In [170]:
def compute_all_metrics(error_precision_spellchecker_function,
                        lexical_precision_spellchecker_function,
                        error_precision_original_word_list,
                        error_precision_answer_word_list,
                        lexical_precision_original_word_list):

    error_precision_result = error_precision_spellchecker_function()
    error_precision_elapsed_time = error_precision_result["elapsed"]
    error_precision_corrected_word_list = error_precision_result["corrected_word_list"]
    error_precision = compute_error_precision(error_precision_original_word_list, error_precision_corrected_word_list, error_precision_answer_word_list)

    lexical_precision_result = lexical_precision_spellchecker_function()
    lexical_precision_elapsed_time = lexical_precision_result["elapsed"]
    lexical_precision_corrected_word_list = lexical_precision_result["corrected_word_list"]
    lexical_precision = compute_lexical_precision(lexical_precision_original_word_list, lexical_precision_corrected_word_list)

    return {"words_per_minute": 60 /
                                ((error_precision_elapsed_time + lexical_precision_elapsed_time) /
                                 (len(error_precision_original_word_list) + len(lexical_precision_original_word_list))),
            "error_precision": error_precision,
            "lexical_precision": lexical_precision,
            "overall_precision": (error_precision + lexical_precision) / 2.0
            # "error_precision_original_word_list": error_precision_original_word_list,
            # "error_precision_corrected_word_list": error_precision_corrected_word_list,
            # "error_precision_answer_word_list": error_precision_answer_word_list,
            # "lexical_precision_original_word_list":lexical_precision_original_word_list
            }

# Spellchecker prototype initialization

In [171]:
!pip install pymorphy2 
!pip install gensim
!pip install python-levenshtein
!pip install stringdist

# Test SpellChecker Prototype v.2

In [172]:
from spellchecker_prototype_v2.spell_checker import SpellChecker

Download models (all files) from [link](https://drive.google.com/drive/folders/1ubVgiIC2pqDOpA-CbjEqV8Jo8uBbk0qi?usp=sharing) and move it to `data/spellchecker_prototype_v2/models` folder

In [173]:
def spellchecker_prototype_v2_correct_func(input_word_list):
    spellchecker_prototype = SpellChecker()
    input_text_for_prototype = " ".join(input_word_list)
    timer = tqdm()
    corrected_word_list = spellchecker_prototype.correct_words(input_text_for_prototype)
    return {"elapsed": timer.format_dict["elapsed"], "corrected_word_list": corrected_word_list}

spellchecker_prototype_v2_metrics = compute_all_metrics(
    lambda : spellchecker_prototype_v2_correct_func(error_precision_test_word_list),
    lambda : spellchecker_prototype_v2_correct_func(lexical_precision_test_word_list),
    error_precision_test_word_list, error_precision_test_answers, lexical_precision_test_word_list)
spellchecker_prototype_v2_metrics

/home/Dmitry.Pogrebnoy/Desktop/MedSpellChecker/spellchecker_prototype_v2


0it [00:00, ?it/s]

['эктренном', 'синусовго', 'даным', 'шнутография', 'сникопальных', 'посление', 'ходавый', 'анмнезе', 'сентированный', 'самоч', 'перевдена', 'стентированеим', 'кардиол', 'переведна', 'догосптальном', 'миним', 'субфебриллитет', 'загрудиные', 'перферия', 'течени', 'оклюзия', 'жнщина', 'пциент', 'зарегистирована', 'сегмета', 'клекте', 'эпид', 'интелектуально', 'нижн', 'прводилась', 'неуст', 'госптализирован', 'вперые', 'послеоперационом', 'госпитализирова', 'ухудщение', 'рентг', 'дальнешем', 'троботическая', 'многосоудистое', 'миокрада', 'сопрвождались', 'дискмфорт', 'потлтвостью', 'левогожелудочка', 'рание', 'виед', 'ангинозныеболи', 'догоситальном', 'ввполнен', 'чаосв', 'октазалась', 'линика', 'ммента', 'проведеной', 'выполненяи', 'леченяи', 'обтилась', 'повтороно', 'зарегистриованы', 'отицает', 'дальнейщем', 'болисохраняются', 'частичынм', 'атерокслеротичсекая', 'деньс', 'назначаля', 'проведеняи', 'беспакоют', 'гипертензи', 'воленйбол', 'болезн', 'стенокардаа', 'продолжалсь', 'поворное'


0it [00:00, ?it/s]
1it [00:23, 23.62s/it]

Fast Text Exception for эктренном


...


174it [15:53,  8.66s/it]

Fast Text Exception for задненижний



200it [16:27,  4.94s/it]
0it [16:27, ?it/s]

Fast Text Exception for формирующийся
Lexical precision
original_word_list --- corrected_word
выборгский --- выборгский
отказываться --- отказываться
нитропрепарат --- биопрепарат
предсердие --- предсердие
диссоциация --- диссоциация
кольцевой --- кольцевой
отрыв --- отрыв
вовлечение --- вовлечение
мелкопузырчатый --- мелкопузырчатый
госпитализировать --- госпитализировать
фтизиатр --- фтизиатр
проксимальный --- проксимальный
приступ --- приступ
рецидивировать --- рецидивировать
беспокоить --- беспокоить
значимый --- значимый
одномоментный --- одномоментный
течение --- течение
постепенной --- постепенный
трафика --- трафила
левый --- левый
область --- область
окклюзия --- окклюзия
каждый --- каждый
аторвастатина --- аторвастатин
проникающий --- проникающий
слух --- слух
отрицательный --- отрицательный
заднее --- заднее
диафрагмальный --- диафрагмальный
физическая --- физическая
реваскуляризация --- реваскуляризация
очевидный --- очевидный
умеренный --- умеренный
самостоятельный --- сам

{'words_per_minute': 4.270933148029589,
 'error_precision': 0.41,
 'lexical_precision': 0.83,
 'overall_precision': 0.62}

# Test Symspellpy - https://github.com/mammothb/symspellpy

In [174]:
!pip install -U symspellpy

In [175]:
!pip show symspellpy

Name: symspellpy
Version: 6.7.5
Summary: Python SymSpell
Home-page: https://github.com/mammothb/symspellpy
Author: mmb L
Author-email: 
License: MIT
Location: /home/Dmitry.Pogrebnoy/Desktop/MedSpellChecker/venv/lib/python3.9/site-packages
Requires: editdistpy
Required-by: 


### Test Symspellpy lookup on basic dict

In [176]:
basic_frequency_dict = '../data/symspell/ru-100k.txt'

In [177]:
from symspellpy import SymSpell, Verbosity

def test_symspell_py_lookup(frequency_dict_path, input_word_list):
    sym_spell_py = SymSpell()
    sym_spell_py.load_dictionary(frequency_dict_path, 0, 1, encoding="UTF8")

    result = []
    timer = tqdm(input_word_list)
    for word in timer:
        suggestions = sym_spell_py.lookup(word, Verbosity.TOP, max_edit_distance=2, include_unknown=True)
        result.append(suggestions[0].term)
    return  {"elapsed" : timer.format_dict["elapsed"], "corrected_word_list" : result}

In [178]:
test_symspell_py_lookup_result = compute_all_metrics(
    lambda : test_symspell_py_lookup(basic_frequency_dict, error_precision_test_word_list),
    lambda : test_symspell_py_lookup(basic_frequency_dict, lexical_precision_test_word_list),
    error_precision_test_word_list, error_precision_test_answers, lexical_precision_test_word_list)
test_symspell_py_lookup_result

100%|██████████| 200/200 [00:00<00:00, 14724.35it/s]


Error precision
original_word_list --- corrected_word --- answer_word_list
эктренном --- коренном --- ['экстренном']
синусовго --- синусовго --- ['синусового']
даным --- данным --- ['данным', 'данный', 'данные']
шнутография --- шнутография --- ['шунтография']
сникопальных --- сникопальных --- ['синкопальных']
посление --- последние --- ['поселение', 'последний']
ходавый --- подавай --- ['ходовый']
анмнезе --- анамнезе --- ['анамнезе', 'анамнез']
сентированный --- сентированный --- ['стентированный']
самоч --- самой --- ['самочувствие']
перевдена --- переведена --- ['переведена']
стентированеим --- стентированеим --- ['стентированием', 'стентирование']
кардиол --- кардинал --- ['кардиолог,кардиологический']
переведна --- переведена --- ['переведена']
догосптальном --- догосптальном --- ['догоспитальном']
миним --- синим --- ['минимум', 'минимальный']
субфебриллитет --- субфебриллитет --- ['субфебрилитет']
загрудиные --- загрудиные --- ['загрудинные', 'загрудинный']
перферия --- перифери

100%|██████████| 200/200 [00:00<00:00, 21991.37it/s]

Lexical precision
original_word_list --- corrected_word
выборгский --- выборгский
отказываться --- отказываться
нитропрепарат --- нитропрепарат
предсердие --- предсердия
диссоциация --- диссоциация
кольцевой --- кольцевой
отрыв --- отрыв
вовлечение --- вовлечение
мелкопузырчатый --- мелкопузырчатый
госпитализировать --- госпитализировать
фтизиатр --- фтизиатр
проксимальный --- проксимальной
приступ --- приступ
рецидивировать --- рецидивировать
беспокоить --- беспокоить
значимый --- значимый
одномоментный --- одномоментный
течение --- течение
постепенной --- постепенной
трафика --- графика
левый --- левый
область --- область
окклюзия --- окклюзия
каждый --- каждый
аторвастатина --- аторвастатина
проникающий --- проникающих
слух --- слух
отрицательный --- отрицательный
заднее --- заднее
диафрагмальный --- диафрагмальный
физическая --- физическая
реваскуляризация --- реваскуляризация
очевидный --- очевидный
умеренный --- умеренный
самостоятельный --- самостоятельный
визуализация --- визуа

{'words_per_minute': 1027145.0465802067,
 'error_precision': 0.42,
 'lexical_precision': 0.78,
 'overall_precision': 0.6}

In [179]:
def test_symspell_py_lookup_compound(frequency_dict_path, input_word_list):
    sym_spell_py = SymSpell()
    sym_spell_py.load_dictionary(frequency_dict_path, 0, 1, encoding="UTF8")

    result = []
    timer = tqdm(input_word_list)
    for word in timer:
        suggestions = sym_spell_py.lookup_compound(word, max_edit_distance=2)
        result.append(suggestions[0].term)
    return {"elapsed" : timer.format_dict["elapsed"], "corrected_word_list" : result}

In [180]:
test_symspell_py_lookup_compound_result = compute_all_metrics(
    lambda : test_symspell_py_lookup_compound(basic_frequency_dict, error_precision_test_word_list),
    lambda : test_symspell_py_lookup_compound(basic_frequency_dict, lexical_precision_test_word_list),
    error_precision_test_word_list, error_precision_test_answers, lexical_precision_test_word_list)
test_symspell_py_lookup_compound_result

100%|██████████| 200/200 [00:00<00:00, 742.66it/s]


Error precision
original_word_list --- corrected_word --- answer_word_list
эктренном --- коренном --- ['экстренном']
синусовго --- синус его --- ['синусового']
даным --- данным --- ['данным', 'данный', 'данные']
шнутография --- и фотография --- ['шунтография']
сникопальных --- они локальных --- ['синкопальных']
посление --- последние --- ['поселение', 'последний']
ходавый --- ход вый --- ['ходовый']
анмнезе --- анамнезе --- ['анамнезе', 'анамнез']
сентированный --- центров данный --- ['стентированный']
самоч --- самой --- ['самочувствие']
перевдена --- переведена --- ['переведена']
стентированеим --- стен титрованием --- ['стентированием', 'стентирование']
кардиол --- кардинал --- ['кардиолог,кардиологический']
переведна --- переведена --- ['переведена']
догосптальном --- до остальном --- ['догоспитальном']
миним --- синим --- ['минимум', 'минимальный']
субфебриллитет --- себе бриллиант --- ['субфебрилитет']
загрудиные --- за рудные --- ['загрудинные', 'загрудинный']
перферия --- периф

100%|██████████| 200/200 [00:00<00:00, 1305.65it/s]

Lexical precision
original_word_list --- corrected_word
выборгский --- выборгский
отказываться --- отказываться
нитропрепарат --- нит препарат
предсердие --- предсердия
диссоциация --- диссоциация
кольцевой --- кольцевой
отрыв --- отрыв
вовлечение --- вовлечение
мелкопузырчатый --- мелкому зубчатый
госпитализировать --- госпитали зимовать
фтизиатр --- эти знать
проксимальный --- проксимальной
приступ --- приступ
рецидивировать --- рецидив кровать
беспокоить --- беспокоить
значимый --- значимый
одномоментный --- одном местный
течение --- течение
постепенной --- постепенной
трафика --- графика
левый --- левый
область --- область
окклюзия --- к ключи
каждый --- каждый
аторвастатина --- автор сталина
проникающий --- проникающих
слух --- слух
отрицательный --- отрицательный
заднее --- заднее
диафрагмальный --- диафрагма юный
физическая --- физическая
реваскуляризация --- рева секуляризации
очевидный --- очевидный
умеренный --- умеренный
самостоятельный --- самостоятельный
визуализация --- в

{'words_per_minute': 56687.79540351705,
 'error_precision': 0.46,
 'lexical_precision': 0.54,
 'overall_precision': 0.5}

# Test SymSpellCppPy - https://github.com/viig99/SymSpellCppPy

In [181]:
!pip install wheel setuptools

In [182]:
!pip install --upgrade SymSpellCppPy

  Using cached SymSpellCppPy-0.0.14.tar.gz (5.4 MB)
  Preparing metadata (setup.py) ... done
  ERROR: Command errored out with exit status 1:
   command: /home/Dmitry.Pogrebnoy/Desktop/MedSpellChecker/venv/bin/python -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-chft0qs5/symspellcpppy_b5e700d3ba054a50a913c2a50c0831e9/setup.py'"'"'; __file__='"'"'/tmp/pip-install-chft0qs5/symspellcpppy_b5e700d3ba054a50a913c2a50c0831e9/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d /tmp/pip-wheel-xmj0nd7g
       cwd: /tmp/pip-install-chft0qs5/symspellcpppy_b5e700d3ba054a50a913c2a50c0831e9/
  Complete output (45 lines):
  running bdist_wheel
  running build
  running build_ext
  Traceback (most recent call last):
    File "/tmp/pi

There was a problem with installing the package. It is still unclear what the reason is. However, this spellchecker should not provide more accuracy, only higher speed since it is implemented in C++. However, the speed of the Python implementation is still enough at this stage.  So I decided to leave it here for now and come back if there is a need for more speed.

# Test JumSpell - https://github.com/bakwc/JamSpell

#### Attention No Windows 10 support!

Not working with compound words

In [183]:
!pip install -I --no-cache-dir jamspell

     |████████████████████████████████| 174 kB 515 kB/s            
  Preparing metadata (setup.py) ... done
  Created wheel for jamspell: filename=jamspell-0.0.12-cp39-cp39-linux_x86_64.whl size=1901056 sha256=5534edc2d15cbc87cf7597a9c73801dc87fd7edda429149526cf1ad8c8ad955a
  Stored in directory: /tmp/pip-ephem-wheel-cache-m77v1r54/wheels/fa/7e/8f/c23ae02f1556243d33cc2e3bddf98f8cc146e174e5855788b6
Successfully built jamspell


In [184]:
!pip show jamspell

Name: jamspell
Version: 0.0.12
Summary: spell checker
Home-page: https://github.com/bakwc/JamSpell
Author: Filipp Ozinov
Author-email: fippo@mail.ru
License: UNKNOWN
Location: /home/Dmitry.Pogrebnoy/Desktop/MedSpellChecker/venv/lib/python3.9/site-packages
Requires: 
Required-by: 


In [185]:
from jamspell import TSpellCorrector
jumspell_model_lib = "../data/junspell/ru_small.bin"

In [186]:
def test_jumspell(jumspell_model_lib, input_word_list):
    jumspell = TSpellCorrector()
    jumspell.LoadLangModel(jumspell_model_lib)

    result = []
    timer = tqdm(input_word_list)
    for word in timer:
        suggestions = jumspell.FixFragment(word)
        result.append(suggestions)
    return  {"elapsed" : timer.format_dict["elapsed"], "corrected_word_list" : result}

In [187]:
test_jumspell_result = compute_all_metrics(
    lambda : test_jumspell(jumspell_model_lib, error_precision_test_word_list),
    lambda : test_jumspell(jumspell_model_lib, lexical_precision_test_word_list),
    error_precision_test_word_list, error_precision_test_answers, lexical_precision_test_word_list)
test_jumspell_result

100%|██████████| 200/200 [00:00<00:00, 1515.91it/s]


Error precision
original_word_list --- corrected_word --- answer_word_list
эктренном --- экстренном --- ['экстренном']
синусовго --- синусовго --- ['синусового']
даным --- даным --- ['данным', 'данный', 'данные']
шнутография --- шнутография --- ['шунтография']
сникопальных --- сникопальных --- ['синкопальных']
посление --- последние --- ['поселение', 'последний']
ходавый --- ходовой --- ['ходовый']
анмнезе --- анмнезе --- ['анамнезе', 'анамнез']
сентированный --- сентированный --- ['стентированный']
самоч --- самом --- ['самочувствие']
перевдена --- переведена --- ['переведена']
стентированеим --- стентированеим --- ['стентированием', 'стентирование']
кардиол --- кардинал --- ['кардиолог,кардиологический']
переведна --- переведена --- ['переведена']
догосптальном --- догосптальном --- ['догоспитальном']
миним --- мини --- ['минимум', 'минимальный']
субфебриллитет --- субфебриллитет --- ['субфебрилитет']
загрудиные --- загрудиные --- ['загрудинные', 'загрудинный']
перферия --- периферия

100%|██████████| 200/200 [00:00<00:00, 3055.26it/s]

Lexical precision
original_word_list --- corrected_word
выборгский --- выборгский
отказываться --- отказываться
нитропрепарат --- нитропрепарат
предсердие --- предсердие
диссоциация --- диссоциация
кольцевой --- кольцевой
отрыв --- отрыв
вовлечение --- вовлечение
мелкопузырчатый --- мелкопузырчатый
госпитализировать --- госпитализировать
фтизиатр --- фтизиатр
проксимальный --- проксимальный
приступ --- приступ
рецидивировать --- рецидивировать
беспокоить --- беспокоить
значимый --- значимый
одномоментный --- одномоментный
течение --- течение
постепенной --- постепенной
трафика --- трафика
левый --- левый
область --- область
окклюзия --- окклюзия
каждый --- каждый
аторвастатина --- аторвастатина
проникающий --- проникающий
слух --- слух
отрицательный --- отрицательный
заднее --- заднее
диафрагмальный --- диафрагмальный
физическая --- физическая
реваскуляризация --- реваскуляризация
очевидный --- очевидный
умеренный --- умеренный
самостоятельный --- самостоятельный
визуализация --- визуа

{'words_per_minute': 120948.54388296275,
 'error_precision': 0.395,
 'lexical_precision': 0.925,
 'overall_precision': 0.66}

# Test Language-tool (language-tool-python) - https://github.com/jxmorris12/language_tool_python

In [188]:
!pip install language-tool-python

In [189]:
!pip show language-tool-python

Name: language-tool-python
Version: 2.6.2
Summary: Checks grammar using LanguageTool.
Home-page: https://github.com/jxmorris12/language_tool_python
Author: Jack Morris
Author-email: jxmorris12@gmail.com
License: GNU GPL
Location: /home/Dmitry.Pogrebnoy/Desktop/MedSpellChecker/venv/lib/python3.9/site-packages
Requires: requests, tqdm
Required-by: 


In [190]:
from language_tool_python import LanguageTool

In [191]:
def test_language_tool(input_word_list):
    tool = LanguageTool('ru')
    result = []
    timer = tqdm(input_word_list)
    for word in timer:
        suggestions = tool.correct(word)
        result.append(suggestions)
    return  {"elapsed" : timer.format_dict["elapsed"], "corrected_word_list" : result}

In [192]:
test_language_tool_result = compute_all_metrics(
    lambda : test_language_tool(error_precision_test_word_list),
    lambda : test_language_tool(lexical_precision_test_word_list),
    error_precision_test_word_list, error_precision_test_answers, lexical_precision_test_word_list)
test_language_tool_result

100%|██████████| 200/200 [00:12<00:00, 16.61it/s]


Error precision
original_word_list --- corrected_word --- answer_word_list
эктренном --- экстренном --- ['экстренном']
синусовго --- синусового --- ['синусового']
даным --- данным --- ['данным', 'данный', 'данные']
шнутография --- фотография --- ['шунтография']
сникопальных --- сник опальных --- ['синкопальных']
посление --- послание --- ['поселение', 'последний']
ходавый --- ходовой --- ['ходовый']
анмнезе --- анамнезе --- ['анамнезе', 'анамнез']
сентированный --- сен тированный --- ['стентированный']
самоч --- само --- ['самочувствие']
перевдена --- передвину --- ['переведена']
стентированеим --- агентированием --- ['стентированием', 'стентирование']
кардиол --- кардио --- ['кардиолог,кардиологический']
переведна --- переведена --- ['переведена']
догосптальном --- догоспитальном --- ['догоспитальном']
миним --- менем --- ['минимум', 'минимальный']
субфебриллитет --- субфебрилитет --- ['субфебрилитет']
загрудиные --- загруженные --- ['загрудинные', 'загрудинный']
перферия --- перифери

100%|██████████| 200/200 [00:12<00:00, 15.73it/s]

Lexical precision
original_word_list --- corrected_word
выборгский --- выборгский
отказываться --- отказываться
нитропрепарат --- биопрепарат
предсердие --- предсердие
диссоциация --- диссоциация
кольцевой --- кольцевой
отрыв --- отрыв
вовлечение --- вовлечение
мелкопузырчатый --- мелкопузырчатый
госпитализировать --- госпитализировать
фтизиатр --- фтизиатр
проксимальный --- проксимальный
приступ --- приступ
рецидивировать --- рецензировать
беспокоить --- беспокоить
значимый --- значимый
одномоментный --- одномоментный
течение --- течение
постепенной --- постепенной
трафика --- трафика
левый --- левый
область --- область
окклюзия --- окклюзия
каждый --- каждый
аторвастатина --- аторвастатина
проникающий --- проникающий
слух --- слух
отрицательный --- отрицательный
заднее --- заднее
диафрагмальный --- диафрагмальный
физическая --- физическая
реваскуляризация --- ре васкуляризация
очевидный --- очевидный
умеренный --- умеренный
самостоятельный --- самостоятельный
визуализация --- визуали

{'words_per_minute': 969.7147369137158,
 'error_precision': 0.64,
 'lexical_precision': 0.845,
 'overall_precision': 0.7424999999999999}

# PySpelling (wrapper for Aspell and Hunspell) - https://facelessuser.github.io/pyspelling/

It is necessary to install for Linux
`sudo apt-get install aspell aspell-ru`
`sudo apt-get install hunspell hunspell-ru`

In [193]:
!pip install pyspelling

This is a console module, so it will be quite difficult to parse the output of the tool and get metrics.

In [194]:
!pyspelling --version

pyspelling 2.7.3


# Aspell wrapper for python - https://pypi.org/project/aspell-python-py3/

It is necessary to install for Linux
`sudo apt-get install libaspell-dev`

In [195]:
!pip install aspell-python-py3

In [196]:
!pip show aspell-python-py3

Name: aspell-python-py3
Version: 1.15
Summary: Wrapper around GNU Aspell for Python 3
Home-page: http://github.com/WojciechMula/aspell-python
Author: Wojciech Muła
Author-email: wojciech_mula@poczta.onet.pl
License: BSD (3 clauses)
Location: /home/Dmitry.Pogrebnoy/Desktop/MedSpellChecker/venv/lib/python3.9/site-packages
Requires: 
Required-by: 


In [197]:
from aspell import Speller

In [198]:
def test_aspell(input_word_list):
    speller = Speller(("lang", "ru"))
    result = []
    timer = tqdm(input_word_list)
    for word in timer:
        suggestions = speller.suggest(word)
        if len(suggestions) == 0:
            result.append(word)
        else:
            result.append(suggestions[0])
    return  {"elapsed" : timer.format_dict["elapsed"], "corrected_word_list" : result}

In [199]:
test_aspell_result = compute_all_metrics(
    lambda : test_aspell(error_precision_test_word_list),
    lambda : test_aspell(lexical_precision_test_word_list),
    error_precision_test_word_list, error_precision_test_answers, lexical_precision_test_word_list)
test_aspell_result

100%|██████████| 200/200 [00:00<00:00, 324.29it/s]


Error precision
original_word_list --- corrected_word --- answer_word_list
эктренном --- экстренном --- ['экстренном']
синусовго --- синусов го --- ['синусового']
даным --- данным --- ['данным', 'данный', 'данные']
шнутография --- нотография --- ['шунтография']
сникопальных --- сник опальных --- ['синкопальных']
посление --- поселение --- ['поселение', 'последний']
ходавый --- ходовой --- ['ходовый']
анмнезе --- анамнезе --- ['анамнезе', 'анамнез']
сентированный --- смонтированный --- ['стентированный']
самоч --- само --- ['самочувствие']
перевдена --- переведена --- ['переведена']
стентированеим --- смонтированном --- ['стентированием', 'стентирование']
кардиол --- кар диол --- ['кардиолог,кардиологический']
переведна --- переведена --- ['переведена']
догосптальном --- госпитальном --- ['догоспитальном']
миним --- мним --- ['минимум', 'минимальный']
субфебриллитет --- субфебриллитет --- ['субфебрилитет']
загрудиные --- загородные --- ['загрудинные', 'загрудинный']
перферия --- перифер

100%|██████████| 200/200 [00:00<00:00, 361.58it/s]

Lexical precision
original_word_list --- corrected_word
выборгский --- выборгский
отказываться --- отказываться
нитропрепарат --- нитропрепарат
предсердие --- предсердие
диссоциация --- диссоциация
кольцевой --- кольцевой
отрыв --- отрыв
вовлечение --- вовлечение
мелкопузырчатый --- мелко пузырчатый
госпитализировать --- госпитализировать
фтизиатр --- фтизиатр
проксимальный --- проксимальный
приступ --- приступ
рецидивировать --- разворовать
беспокоить --- беспокоить
значимый --- значимый
одномоментный --- одномоментный
течение --- течение
постепенной --- постепенной
трафика --- трафика
левый --- левый
область --- область
окклюзия --- окклюзия
каждый --- каждый
аторвастатина --- отрастанию
проникающий --- проникающий
слух --- слух
отрицательный --- отрицательный
заднее --- заднее
диафрагмальный --- поддиафрагмальный
физическая --- физическая
реваскуляризация --- секуляризация
очевидный --- очевидный
умеренный --- умеренный
самостоятельный --- самостоятельный
визуализация --- визуализац

{'words_per_minute': 20500.240818011436,
 'error_precision': 0.65,
 'lexical_precision': 0.775,
 'overall_precision': 0.7125}

# Test Hunspell wrapper for Python - https://pypi.org/project/hunspell/

It is necessary to install for Linux
`sudo apt-get install libhunspell-dev`

In [200]:
!pip install hunspell

In [201]:
!pip show hunspell

Name: hunspell
Version: 0.5.5
Summary: Module for the Hunspell spellchecker engine
Home-page: http://github.com/blatinier/pyhunspell
Author: Benoît Latinier
Author-email: benoit@latinier.fr
License: LGPLv3
Location: /home/Dmitry.Pogrebnoy/Desktop/MedSpellChecker/venv/lib/python3.9/site-packages
Requires: 
Required-by: 


In [202]:
from hunspell import HunSpell

In [203]:
def test_hunspell(input_word_list):
    speller = HunSpell('../data/hunspell/index.dic', '../data/hunspell/index.aff')
    result = []
    timer = tqdm(input_word_list)
    for word in timer:
        suggestions = speller.suggest(word)
        if len(suggestions) == 0:
            result.append(word)
        else:
            result.append(suggestions[0])
    return  {"elapsed" : timer.format_dict["elapsed"], "corrected_word_list" : result}

In [204]:
test_hunspell_result = compute_all_metrics(
    lambda : test_hunspell(error_precision_test_word_list),
    lambda : test_hunspell(lexical_precision_test_word_list),
    error_precision_test_word_list, error_precision_test_answers, lexical_precision_test_word_list)
test_hunspell_result

100%|██████████| 200/200 [00:15<00:00, 13.19it/s]


Error precision
original_word_list --- corrected_word --- answer_word_list
эктренном --- экстренном --- ['экстренном']
синусовго --- синусов го --- ['синусового']
даным --- даны --- ['данным', 'данный', 'данные']
шнутография --- нотография --- ['шунтография']
сникопальных --- сник опальных --- ['синкопальных']
посление --- поселение --- ['поселение', 'последний']
ходавый --- хода вый --- ['ходовый']
анмнезе --- анамнезе --- ['анамнезе', 'анамнез']
сентированный --- сориентированный --- ['стентированный']
самоч --- само --- ['самочувствие']
перевдена --- перевидена --- ['переведена']
стентированеим --- сцементирован --- ['стентированием', 'стентирование']
кардиол --- кар диол --- ['кардиолог,кардиологический']
переведна --- переведена --- ['переведена']
догосптальном --- госпитальном --- ['догоспитальном']
миним --- мини --- ['минимум', 'минимальный']
субфебриллитет --- субфебрилитет --- ['субфебрилитет']
загрудиные --- нагрудные --- ['загрудинные', 'загрудинный']
перферия --- периферия

100%|██████████| 200/200 [00:17<00:00, 11.58it/s]

Lexical precision
original_word_list --- corrected_word
выборгский --- выборгски
отказываться --- отказываться
нитропрепарат --- органопрепарат
предсердие --- предсердие
диссоциация --- диссоциация
кольцевой --- кольцевое
отрыв --- отрыва
вовлечение --- вовлечение
мелкопузырчатый --- мелко пузырчатый
госпитализировать --- госпитализировать
фтизиатр --- фтизиатр
проксимальный --- проксимальной
приступ --- приступа
рецидивировать --- рецитировать
беспокоить --- беспокоить
значимый --- значимы
одномоментный --- одномоментный
течение --- течение
постепенной --- постепенной
трафика --- трафика
левый --- левы
область --- областью
окклюзия --- окклюзия
каждый --- каждой
аторвастатина --- настораживать
проникающий --- поникающий
слух --- сух
отрицательный --- отрицательны
заднее --- заднее
диафрагмальный --- диафрагмальный
физическая --- физическая
реваскуляризация --- секуляризация
очевидный --- очевидны
умеренный --- умеренный
самостоятельный --- самостоятельный
визуализация --- визуализация

{'words_per_minute': 739.8082495384857,
 'error_precision': 0.59,
 'lexical_precision': 0.49,
 'overall_precision': 0.54}

# Pyspellchecker (Piter Norwig's algorithm) - https://github.com/barrust/pyspellchecker

In [205]:
!pip install pyspellchecker

In [206]:
!pip show pyspellchecker

Name: pyspellchecker
Version: 0.6.2
Summary: Pure python spell checker based on work by Peter Norvig
Home-page: https://github.com/barrust/pyspellchecker
Author: Tyler Barrus
Author-email: barrust@gmail.com
License: MIT
Location: /home/Dmitry.Pogrebnoy/Desktop/MedSpellChecker/venv/lib/python3.9/site-packages
Requires: 
Required-by: 


In [207]:
import spellchecker

In [208]:
def test_pyspellshecker(input_word_list):
    speller = spellchecker.SpellChecker(language='ru')
    result = []
    timer = tqdm(input_word_list)
    for word in timer:
        suggestions = speller.correction(word)
        result.append(suggestions)
    return  {"elapsed" : timer.format_dict["elapsed"], "corrected_word_list" : result}

In [209]:
test_pyspellshecker_result = compute_all_metrics(
    lambda : test_pyspellshecker(error_precision_test_word_list),
    lambda : test_pyspellshecker(lexical_precision_test_word_list),
    error_precision_test_word_list, error_precision_test_answers, lexical_precision_test_word_list)
test_pyspellshecker_result

100%|██████████| 200/200 [00:45<00:00,  4.38it/s]


Error precision
original_word_list --- corrected_word --- answer_word_list
эктренном --- экстренное --- ['экстренном']
синусовго --- синусовый --- ['синусового']
даным --- давным --- ['данным', 'данный', 'данные']
шнутография --- шнутография --- ['шунтография']
сникопальных --- сникопальных --- ['синкопальных']
посление --- последние --- ['поселение', 'последний']
ходавый --- подавай --- ['ходовый']
анмнезе --- анмнезе --- ['анамнезе', 'анамнез']
сентированный --- сентированный --- ['стентированный']
самоч --- самое --- ['самочувствие']
перевдена --- переведено --- ['переведена']
стентированеим --- стентированеим --- ['стентированием', 'стентирование']
кардиол --- кардинал --- ['кардиолог,кардиологический']
переведна --- переведено --- ['переведена']
догосптальном --- догосптальном --- ['догоспитальном']
миним --- мини --- ['минимум', 'минимальный']
субфебриллитет --- субфебриллитет --- ['субфебрилитет']
загрудиные --- загрудиные --- ['загрудинные', 'загрудинный']
перферия --- перферия

100%|██████████| 200/200 [00:45<00:00,  4.39it/s]

Lexical precision
original_word_list --- corrected_word
выборгский --- выборгский
отказываться --- отказываться
нитропрепарат --- нитропрепарат
предсердие --- предсердие
диссоциация --- ассоциация
кольцевой --- кольцевой
отрыв --- отрыв
вовлечение --- вовлечение
мелкопузырчатый --- мелкопузырчатый
госпитализировать --- госпитализировать
фтизиатр --- фтизиатр
проксимальный --- проксимальный
приступ --- приступ
рецидивировать --- рецидивировать
беспокоить --- беспокоить
значимый --- знакомый
одномоментный --- одномоментный
течение --- течение
постепенной --- постепенно
трафика --- графика
левый --- левый
область --- область
окклюзия --- окклюзия
каждый --- каждый
аторвастатина --- аторвастатина
проникающий --- проникающие
слух --- слух
отрицательный --- отрицательный
заднее --- заднее
диафрагмальный --- диафрагмальный
физическая --- физическая
реваскуляризация --- реваскуляризация
очевидный --- очевидный
умеренный --- уверенный
самостоятельный --- самостоятельно
визуализация --- визуализ

{'words_per_minute': 263.21813838563963,
 'error_precision': 0.335,
 'lexical_precision': 0.765,
 'overall_precision': 0.55}

# PyEnchant - https://pypi.org/project/pyenchant/

In [210]:
!pip install pyenchant

In [211]:
!pip show pyenchant

Name: pyenchant
Version: 3.2.2
Summary: Python bindings for the Enchant spellchecking system
Home-page: https://pyenchant.github.io/pyenchant/
Author: Dimitri Merejkowsky
Author-email: d.merej@gmail.com
License: LGPL
Location: /home/Dmitry.Pogrebnoy/Desktop/MedSpellChecker/venv/lib/python3.9/site-packages
Requires: 
Required-by: 


In [212]:
import enchant

In [213]:
def test_enchant(input_word_list):
    speller = enchant.Dict("ru")
    result = []
    timer = tqdm(input_word_list)
    for word in timer:
        suggestions = speller.suggest(word)
        if len(suggestions) == 0:
            result.append(word)
        else:
            result.append(suggestions[0])
    return  {"elapsed" : timer.format_dict["elapsed"], "corrected_word_list" : result}

In [214]:
test_enchant_result = compute_all_metrics(
    lambda : test_enchant(error_precision_test_word_list),
    lambda : test_enchant(lexical_precision_test_word_list),
    error_precision_test_word_list, error_precision_test_answers, lexical_precision_test_word_list)
test_enchant_result

100%|██████████| 200/200 [00:07<00:00, 28.45it/s]


Error precision
original_word_list --- corrected_word --- answer_word_list
эктренном --- экстренном --- ['экстренном']
синусовго --- синусов го --- ['синусового']
даным --- даны --- ['данным', 'данный', 'данные']
шнутография --- нотография --- ['шунтография']
сникопальных --- сник опальных --- ['синкопальных']
посление --- поселение --- ['поселение', 'последний']
ходавый --- хордовый --- ['ходовый']
анмнезе --- анамнезе --- ['анамнезе', 'анамнез']
сентированный --- сориентированный --- ['стентированный']
самоч --- само --- ['самочувствие']
перевдена --- перевидена --- ['переведена']
стентированеим --- сориентировавшим --- ['стентированием', 'стентирование']
кардиол --- кар диол --- ['кардиолог,кардиологический']
переведна --- переведена --- ['переведена']
догосптальном --- госпитальном --- ['догоспитальном']
миним --- мини --- ['минимум', 'минимальный']
субфебриллитет --- субмиллиметровый --- ['субфебрилитет']
загрудиные --- нагрудные --- ['загрудинные', 'загрудинный']
перферия --- пер

100%|██████████| 200/200 [00:07<00:00, 25.07it/s]

Lexical precision
original_word_list --- corrected_word
выборгский --- выборгской
отказываться --- отказываться
нитропрепарат --- биопрепарат
предсердие --- предсердие
диссоциация --- диссоциация
кольцевой --- кольцевое
отрыв --- отрыва
вовлечение --- вовлечение
мелкопузырчатый --- мелко пузырчатый
госпитализировать --- госпитализировать
фтизиатр --- фтизиатр
проксимальный --- проксимальной
приступ --- приступи
рецидивировать --- резецировать
беспокоить --- беспокоить
значимый --- значимы
одномоментный --- одномоментный
течение --- течение
постепенной --- постепенной
трафика --- трафика
левый --- левы
область --- областью
окклюзия --- окклюзия
каждый --- каждой
аторвастатина --- настораживать
проникающий --- поникающий
слух --- сух
отрицательный --- отрицательны
заднее --- заднее
диафрагмальный --- поддиафрагмальный
физическая --- физическая
реваскуляризация --- секуляризация
очевидный --- очевидны
умеренный --- умеренный
самостоятельный --- самостоятельный
визуализация --- визуализаци

{'words_per_minute': 1598.9686624475778,
 'error_precision': 0.6,
 'lexical_precision': 0.455,
 'overall_precision': 0.5275}

## Show results in table

In [215]:
!pip install tabulate

In [216]:
from tabulate import tabulate

In [217]:
header = ["Tool name", "Avg words per minute", "Error precision", "Lexical precision", "Overall precision"]
table = [["Spellchecker prototype v2", spellchecker_prototype_v2_metrics["words_per_minute"], spellchecker_prototype_v2_metrics["error_precision"], spellchecker_prototype_v2_metrics["lexical_precision"], spellchecker_prototype_v2_metrics["overall_precision"]],
         ["Symspell lookup", test_symspell_py_lookup_result["words_per_minute"], test_symspell_py_lookup_result["error_precision"], test_symspell_py_lookup_result["lexical_precision"], test_symspell_py_lookup_result["overall_precision"]],
         ["Symspell lookup compound", test_symspell_py_lookup_compound_result["words_per_minute"], test_symspell_py_lookup_compound_result["error_precision"], test_symspell_py_lookup_compound_result["lexical_precision"], test_symspell_py_lookup_compound_result["overall_precision"]],
         ["Jumspell", test_jumspell_result["words_per_minute"], test_jumspell_result["error_precision"], test_jumspell_result["lexical_precision"], test_jumspell_result["overall_precision"]],
         ["LanguageTool", test_language_tool_result["words_per_minute"], test_language_tool_result["error_precision"], test_language_tool_result["lexical_precision"], test_language_tool_result["overall_precision"]],
         ["Aspell", test_aspell_result["words_per_minute"], test_aspell_result["error_precision"], test_aspell_result["lexical_precision"], test_aspell_result["overall_precision"]],
         ["Hunspell", test_hunspell_result["words_per_minute"], test_hunspell_result["error_precision"], test_hunspell_result["lexical_precision"], test_hunspell_result["overall_precision"]],
         ["PySpellChecker (Peter Norvig's)", test_pyspellshecker_result["words_per_minute"], test_pyspellshecker_result["error_precision"], test_pyspellshecker_result["lexical_precision"], test_pyspellshecker_result["overall_precision"]],
         ["Enchant", test_enchant_result["words_per_minute"], test_enchant_result["error_precision"], test_enchant_result["lexical_precision"], test_enchant_result["overall_precision"]],]
print(tabulate(table, headers=header))

Tool name                          Avg words per minute    Error precision    Lexical precision    Overall precision
-------------------------------  ----------------------  -----------------  -------------------  -------------------
Spellchecker prototype v2                   4.27093                  0.41                 0.83                0.62
Symspell lookup                             1.02715e+06              0.42                 0.78                0.6
Symspell lookup compound                56687.8                      0.46                 0.54                0.5
Jumspell                               120949                        0.395                0.925               0.66
LanguageTool                              969.715                    0.64                 0.845               0.7425
Aspell                                  20500.2                      0.65                 0.775               0.7125
Hunspell                                  739.808                    0.59 